# **Botnet Detection in IOT devices using an Autoencoder**

## Loading Required Libraries

In [ ]:
import os
import numpy as np
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras import layers, losses, Sequential
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

## Mounting drive

In [ ]:
from google.colab import drive
drive.mount('/drive')

Mounted at /drive


## Autoencoder

In [ ]:

class Autoencoder(Model):
    def __init__(self):
        super(Autoencoder, self).__init__()
        self.encoder = Sequential([
            layers.Dense(115, activation="relu"),
            layers.Dense(86, activation="relu"),
            layers.Dense(57, activation="relu"),
            layers.Dense(37, activation="relu"),
            layers.Dense(28, activation="relu")
        ])
        self.decoder = Sequential([
            layers.Dense(37, activation="relu"),
            layers.Dense(57, activation="relu"),
            layers.Dense(86, activation="relu"),
            layers.Dense(115, activation="sigmoid")
        ])
    
    def call(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return decoded

## Function To print Stats

In [ ]:
def print_stats(data, outcome):
    print(f"Shape of data: {data.shape}")
    print(f"Detected anomalies: {np.mean(outcome)*100}%")
    print()

## IOT Device 1: Danmini Doorbell

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("1.benign.csv")
X_train = benign[:40000]
X_test0 = benign[40000:]
X_test1 = load_nbaiot("1.mirai.scan.csv")
X_test2 = load_nbaiot("1.mirai.ack.csv")
X_test3 = load_nbaiot("1.mirai.syn.csv")
X_test4 = load_nbaiot("1.mirai.udp.csv")
X_test5 = load_nbaiot("1.mirai.udpplain.csv")
X_test6 = load_nbaiot("1.gafgyt.combo.csv")
X_test7 = load_nbaiot("1.gafgyt.junk.csv")
X_test8 = load_nbaiot("1.gafgyt.scan.csv")
X_test9 = load_nbaiot("1.gafgyt.tcp.csv")
X_test10 = load_nbaiot("1.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(40000, 115) (9548, 115) (107685, 115) (102195, 115) (122573, 115) (237665, 115) (81982, 115) (59718, 115) (29068, 115) (29849, 115) (92141, 115) (105874, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.012), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=800,
    validation_split=0.3,
    shuffle=True,
    # callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/20
875/875 [==============================] - 7s 7ms/step - loss: 0.0024 - val_loss: 0.0013
Epoch 2/20
875/875 [==============================] - 4s 5ms/step - loss: 8.4994e-04 - val_loss: 0.0012
Epoch 3/20
875/875 [==============================] - 4s 5ms/step - loss: 9.1146e-04 - val_loss: 0.0013
Epoch 4/20
875/875 [==============================] - 4s 5ms/step - loss: 7.8008e-04 - val_loss: 0.0011
Epoch 5/20
875/875 [==============================] - 4s 5ms/step - loss: 8.7314e-04 - val_loss: 0.0010
Epoch 6/20
875/875 [==============================] - 4s 5ms/step - loss: 8.2081e-04 - val_loss: 0.0012
Epoch 7/20
875/875 [==============================] - 4s 5ms/step - loss: 8.0725e-04 - val_loss: 9.6966e-04
Epoch 8/20
875/875 [==============================] - 4s 5ms/step - loss: 7.1022e-04 - val_loss: 0.0010
Epoch 9/20
875/875 [==============================] - 4s 5ms/step - loss: 7.4094e-04 - val_loss: 0.0011
Epoch 10/20
875/875 [==============================] - 4s 5ms/st

### Testing the autoencoder

In [ ]:
def predict(x, threshold=0.042, window_size=82):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (9548, 115)
Detected anomalies: 0.0%

1
Shape of data: (107685, 115)
Detected anomalies: 100.0%

2
Shape of data: (102195, 115)
Detected anomalies: 100.0%

3
Shape of data: (122573, 115)
Detected anomalies: 100.0%

4
Shape of data: (237665, 115)
Detected anomalies: 100.0%

5
Shape of data: (81982, 115)
Detected anomalies: 100.0%

6
Shape of data: (59718, 115)
Detected anomalies: 100.0%

7
Shape of data: (29068, 115)
Detected anomalies: 100.0%

8
Shape of data: (29849, 115)
Detected anomalies: 100.0%

9
Shape of data: (92141, 115)
Detected anomalies: 0.0%

10
Shape of data: (105874, 115)
Detected anomalies: 0.0%



## IOT Device 2: Ecobee Thermostat

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("2.benign.csv")
X_train = benign[:10000]
X_test0 = benign[10000:]
X_test1 = load_nbaiot("2.mirai.scan.csv")
X_test2 = load_nbaiot("2.mirai.ack.csv")
X_test3 = load_nbaiot("2.mirai.syn.csv")
X_test4 = load_nbaiot("2.mirai.udp.csv")
X_test5 = load_nbaiot("2.mirai.udpplain.csv")
X_test6 = load_nbaiot("2.gafgyt.combo.csv")
X_test7 = load_nbaiot("2.gafgyt.junk.csv")
X_test8 = load_nbaiot("2.gafgyt.scan.csv")
X_test9 = load_nbaiot("2.gafgyt.tcp.csv")
X_test10 = load_nbaiot("2.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(10000, 115) (3113, 115) (43192, 115) (113285, 115) (116807, 115) (151481, 115) (87368, 115) (53012, 115) (30312, 115) (27494, 115) (95021, 115) (104791, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.028), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=250,
    validation_split=0.3,
    shuffle=True,
    # callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/250
219/219 [==============================] - 2s 5ms/step - loss: 0.0070 - val_loss: 0.0041
Epoch 2/250
219/219 [==============================] - 1s 5ms/step - loss: 0.0036 - val_loss: 0.0019
Epoch 3/250
219/219 [==============================] - 1s 5ms/step - loss: 0.0031 - val_loss: 0.0026
Epoch 4/250
219/219 [==============================] - 1s 5ms/step - loss: 0.0043 - val_loss: 0.0035
Epoch 5/250
219/219 [==============================] - 1s 5ms/step - loss: 0.0043 - val_loss: 0.0033
Epoch 6/250
219/219 [==============================] - 1s 5ms/step - loss: 0.0037 - val_loss: 0.0026
Epoch 7/250
219/219 [==============================] - 1s 5ms/step - loss: 0.0037 - val_loss: 0.0045
Epoch 8/250
219/219 [==============================] - 1s 5ms/step - loss: 0.0058 - val_loss: 0.0042
Epoch 9/250
219/219 [==============================] - 1s 4ms/step - loss: 0.0058 - val_loss: 0.0056
Epoch 10/250
219/219 [==============================] - 1s 5ms/step - loss: 0.0056 - val_lo

### Testing the autoencoder

In [ ]:
def predict(x, threshold=0.011, window_size=20):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (3113, 115)
Detected anomalies: 1.0019392372333549%

1
Shape of data: (43192, 115)
Detected anomalies: 100.0%

2
Shape of data: (113285, 115)
Detected anomalies: 100.0%

3
Shape of data: (116807, 115)
Detected anomalies: 100.0%

4
Shape of data: (151481, 115)
Detected anomalies: 100.0%

5
Shape of data: (87368, 115)
Detected anomalies: 100.0%

6
Shape of data: (53012, 115)
Detected anomalies: 100.0%

7
Shape of data: (30312, 115)
Detected anomalies: 100.0%

8
Shape of data: (27494, 115)
Detected anomalies: 100.0%

9
Shape of data: (95021, 115)
Detected anomalies: 100.0%

10
Shape of data: (104791, 115)
Detected anomalies: 100.0%



## IOT Device 3: Ennio Doorbell

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("3.benign.csv")
X_train = benign[:30000]
X_test0 = benign[30000:]
X_test1 = load_nbaiot("3.gafgyt.combo.csv")
X_test2 = load_nbaiot("3.gafgyt.junk.csv")
X_test3 = load_nbaiot("3.gafgyt.scan.csv")
X_test4 = load_nbaiot("3.gafgyt.tcp.csv")
X_test5 = load_nbaiot("3.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape)

(30000, 115) (9100, 115) (53014, 115) (29797, 115) (28120, 115) (101536, 115) (103933, 115) (53012, 115) (30312, 115) (27494, 115) (95021, 115) (104791, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.003), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=350,
    validation_split=0.3,
    shuffle=True,
    # callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/350
657/657 [==============================] - 4s 5ms/step - loss: 0.0066 - val_loss: 0.0012
Epoch 2/350
657/657 [==============================] - 3s 5ms/step - loss: 0.0014 - val_loss: 9.5363e-04
Epoch 3/350
657/657 [==============================] - 3s 5ms/step - loss: 0.0010 - val_loss: 8.6308e-04
Epoch 4/350
657/657 [==============================] - 3s 5ms/step - loss: 8.3409e-04 - val_loss: 4.2413e-04
Epoch 5/350
657/657 [==============================] - 3s 5ms/step - loss: 7.4637e-04 - val_loss: 0.0011
Epoch 6/350
657/657 [==============================] - 3s 5ms/step - loss: 8.2544e-04 - val_loss: 3.5306e-04
Epoch 7/350
657/657 [==============================] - 3s 5ms/step - loss: 5.5955e-04 - val_loss: 3.3627e-04
Epoch 8/350
657/657 [==============================] - 3s 5ms/step - loss: 5.3242e-04 - val_loss: 3.1785e-04
Epoch 9/350
657/657 [==============================] - 3s 5ms/step - loss: 5.0002e-04 - val_loss: 3.1938e-04
Epoch 10/350
657/657 [=================

### Testing the autoencoder

In [ ]:
def predict(x, threshold=0.011, window_size=22):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (9100, 115)
Detected anomalies: 0.0%

1
Shape of data: (53014, 115)
Detected anomalies: 99.99811295831525%

2
Shape of data: (29797, 115)
Detected anomalies: 100.0%

3
Shape of data: (28120, 115)
Detected anomalies: 100.0%

4
Shape of data: (101536, 115)
Detected anomalies: 0.0%

5
Shape of data: (103933, 115)
Detected anomalies: 0.0%



## IOT Device 4: Philips B120N10 Baby Monitor

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("4.benign.csv")
X_train = benign[:160000]
X_test0 = benign[160000:]
X_test1 = load_nbaiot("4.mirai.scan.csv")
X_test2 = load_nbaiot("4.mirai.ack.csv")
X_test3 = load_nbaiot("4.mirai.syn.csv")
X_test4 = load_nbaiot("4.mirai.udp.csv")
X_test5 = load_nbaiot("4.mirai.udpplain.csv")
X_test6 = load_nbaiot("4.gafgyt.combo.csv")
X_test7 = load_nbaiot("4.gafgyt.junk.csv")
X_test8 = load_nbaiot("4.gafgyt.scan.csv")
X_test9 = load_nbaiot("4.gafgyt.tcp.csv")
X_test10 = load_nbaiot("4.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(160000, 115) (15240, 115) (103621, 115) (91123, 115) (118128, 115) (217034, 115) (80808, 115) (58152, 115) (28349, 115) (27859, 115) (92581, 115) (105782, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.016), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=100,
    validation_split=0.3,
    shuffle=True,
    # callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/100
3500/3500 [==============================] - 17s 5ms/step - loss: 0.0032 - val_loss: 0.0011
Epoch 2/100
3500/3500 [==============================] - 16s 5ms/step - loss: 0.0014 - val_loss: 0.0012
Epoch 3/100
3500/3500 [==============================] - 16s 4ms/step - loss: 0.0049 - val_loss: 0.0036
Epoch 4/100
3500/3500 [==============================] - 17s 5ms/step - loss: 0.0050 - val_loss: 0.0036
Epoch 5/100
3500/3500 [==============================] - 16s 5ms/step - loss: 0.0050 - val_loss: 0.0036
Epoch 6/100
3500/3500 [==============================] - 18s 5ms/step - loss: 0.0050 - val_loss: 0.0036
Epoch 7/100
3500/3500 [==============================] - 16s 5ms/step - loss: 0.0050 - val_loss: 0.0036
Epoch 8/100
3500/3500 [==============================] - 16s 5ms/step - loss: 0.0050 - val_loss: 0.0036
Epoch 9/100
3500/3500 [==============================] - 16s 5ms/step - loss: 0.0050 - val_loss: 0.0036
Epoch 10/100
3500/3500 [==============================] - 15s 4m

### Testing the autoencoder

In [ ]:
def predict(x, threshold=0.030, window_size=65):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (15240, 115)
Detected anomalies: 3.8679493937796523%

1
Shape of data: (103621, 115)
Detected anomalies: 100.0%

2
Shape of data: (91123, 115)
Detected anomalies: 100.0%

3
Shape of data: (118128, 115)
Detected anomalies: 100.0%

4
Shape of data: (217034, 115)
Detected anomalies: 100.0%

5
Shape of data: (80808, 115)
Detected anomalies: 100.0%

6
Shape of data: (58152, 115)
Detected anomalies: 100.0%

7
Shape of data: (28349, 115)
Detected anomalies: 100.0%

8
Shape of data: (27859, 115)
Detected anomalies: 100.0%

9
Shape of data: (92581, 115)
Detected anomalies: 100.0%

10
Shape of data: (105782, 115)
Detected anomalies: 100.0%



## IOT Device 5: Provision PT 737E Security Camera

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("5.benign.csv")
X_train = benign[:55000]
X_test0 = benign[55000:]
X_test1 = load_nbaiot("5.mirai.scan.csv")
X_test2 = load_nbaiot("5.mirai.ack.csv")
X_test3 = load_nbaiot("5.mirai.syn.csv")
X_test4 = load_nbaiot("5.mirai.udp.csv")
X_test5 = load_nbaiot("5.mirai.udpplain.csv")
X_test6 = load_nbaiot("5.gafgyt.combo.csv")
X_test7 = load_nbaiot("5.gafgyt.junk.csv")
X_test8 = load_nbaiot("5.gafgyt.scan.csv")
X_test9 = load_nbaiot("5.gafgyt.tcp.csv")
X_test10 = load_nbaiot("5.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(55000, 115) (7154, 115) (96781, 115) (60554, 115) (65746, 115) (156248, 115) (56681, 115) (61380, 115) (30898, 115) (29297, 115) (104510, 115) (104011, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.026), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=300,
    validation_split=0.3,
    shuffle=True,
    # callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/300
1204/1204 [==============================] - 7s 5ms/step - loss: 0.0064 - val_loss: 0.0044
Epoch 2/300
1204/1204 [==============================] - 6s 5ms/step - loss: 0.0038 - val_loss: 0.0037
Epoch 3/300
1204/1204 [==============================] - 6s 5ms/step - loss: 0.0054 - val_loss: 0.0052
Epoch 4/300
1204/1204 [==============================] - 6s 5ms/step - loss: 0.0046 - val_loss: 0.0049
Epoch 5/300
1204/1204 [==============================] - 5s 5ms/step - loss: 0.0045 - val_loss: 0.0044
Epoch 6/300
1204/1204 [==============================] - 6s 5ms/step - loss: 0.0043 - val_loss: 0.0053
Epoch 7/300
1204/1204 [==============================] - 6s 5ms/step - loss: 0.0044 - val_loss: 0.0044
Epoch 8/300
1204/1204 [==============================] - 6s 5ms/step - loss: 0.0044 - val_loss: 0.0047
Epoch 9/300
1204/1204 [==============================] - 6s 5ms/step - loss: 0.0042 - val_loss: 0.0045
Epoch 10/300
1204/1204 [==============================] - 6s 5ms/step - l

### Testing the autoencoder

In [ ]:
def predict(x, threshold=0.035, window_size=32):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (7154, 115)
Detected anomalies: 21.05854274884178%

1
Shape of data: (96781, 115)
Detected anomalies: 100.0%

2
Shape of data: (60554, 115)
Detected anomalies: 100.0%

3
Shape of data: (65746, 115)
Detected anomalies: 100.0%

4
Shape of data: (156248, 115)
Detected anomalies: 100.0%

5
Shape of data: (56681, 115)
Detected anomalies: 100.0%

6
Shape of data: (61380, 115)
Detected anomalies: 100.0%

7
Shape of data: (30898, 115)
Detected anomalies: 100.0%

8
Shape of data: (29297, 115)
Detected anomalies: 100.0%

9
Shape of data: (104510, 115)
Detected anomalies: 100.0%

10
Shape of data: (104011, 115)
Detected anomalies: 100.0%



## IOT Device 6: Provision PT 838 Security Camera

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("6.benign.csv")
X_train = benign[:90000]
X_test0 = benign[90000:]
X_test1 = load_nbaiot("6.mirai.scan.csv")
X_test2 = load_nbaiot("6.mirai.ack.csv")
X_test3 = load_nbaiot("6.mirai.syn.csv")
X_test4 = load_nbaiot("6.mirai.udp.csv")
X_test5 = load_nbaiot("6.mirai.udpplain.csv")
X_test6 = load_nbaiot("6.gafgyt.combo.csv")
X_test7 = load_nbaiot("6.gafgyt.junk.csv")
X_test8 = load_nbaiot("6.gafgyt.scan.csv")
X_test9 = load_nbaiot("6.gafgyt.tcp.csv")
X_test10 = load_nbaiot("6.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(90000, 115) (8514, 115) (97096, 115) (57997, 115) (61851, 115) (158608, 115) (53785, 115) (57530, 115) (29068, 115) (28397, 115) (89387, 115) (104658, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.008), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=450,
    validation_split=0.3,
    shuffle=True,
    # callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/450
1969/1969 [==============================] - 8s 4ms/step - loss: 0.0030 - val_loss: 0.0013
Epoch 2/450
1969/1969 [==============================] - 6s 3ms/step - loss: 0.0011 - val_loss: 9.3352e-04
Epoch 3/450
1969/1969 [==============================] - 6s 3ms/step - loss: 0.0013 - val_loss: 0.0020
Epoch 4/450
1969/1969 [==============================] - 6s 3ms/step - loss: 0.0014 - val_loss: 0.0014
Epoch 5/450
1969/1969 [==============================] - 5s 3ms/step - loss: 0.0012 - val_loss: 0.0013
Epoch 6/450
1969/1969 [==============================] - 5s 2ms/step - loss: 9.7308e-04 - val_loss: 0.0051
Epoch 7/450
1969/1969 [==============================] - 5s 3ms/step - loss: 0.0013 - val_loss: 0.0014
Epoch 8/450
1969/1969 [==============================] - 5s 2ms/step - loss: 8.2387e-04 - val_loss: 8.5606e-04
Epoch 9/450
1969/1969 [==============================] - 5s 3ms/step - loss: 0.0012 - val_loss: 0.0019
Epoch 10/450
1969/1969 [==============================] -

### Testing the autoencoder

In [ ]:
def predict(x, threshold=0.038, window_size=43):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (8514, 115)
Detected anomalies: 0.0%

1
Shape of data: (97096, 115)
Detected anomalies: 100.0%

2
Shape of data: (57997, 115)
Detected anomalies: 100.0%

3
Shape of data: (61851, 115)
Detected anomalies: 100.0%

4
Shape of data: (158608, 115)
Detected anomalies: 100.0%

5
Shape of data: (53785, 115)
Detected anomalies: 100.0%

6
Shape of data: (57530, 115)
Detected anomalies: 100.0%

7
Shape of data: (29068, 115)
Detected anomalies: 100.0%

8
Shape of data: (28397, 115)
Detected anomalies: 100.0%

9
Shape of data: (89387, 115)
Detected anomalies: 100.0%

10
Shape of data: (104658, 115)
Detected anomalies: 100.0%



## IOT Device 7: Samsung SNH 1011 N Webcam

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("7.benign.csv")
X_train = benign[:50000]
X_test0 = benign[50000:]
X_test1 = load_nbaiot("7.gafgyt.combo.csv")
X_test2 = load_nbaiot("7.gafgyt.junk.csv")
X_test3 = load_nbaiot("7.gafgyt.scan.csv")
X_test4 = load_nbaiot("7.gafgyt.tcp.csv")
X_test5 = load_nbaiot("7.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape)

(50000, 115) (2150, 115) (58669, 115) (28305, 115) (27698, 115) (97783, 115) (110617, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.013), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=150,
    validation_split=0.3,
    shuffle=True,
    # callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/150
1094/1094 [==============================] - 4s 3ms/step - loss: 0.0032 - val_loss: 0.0012
Epoch 2/150
1094/1094 [==============================] - 3s 3ms/step - loss: 0.0018 - val_loss: 3.9257e-04
Epoch 3/150
1094/1094 [==============================] - 3s 3ms/step - loss: 0.0015 - val_loss: 4.0072e-04
Epoch 4/150
1094/1094 [==============================] - 3s 3ms/step - loss: 0.0014 - val_loss: 4.5951e-04
Epoch 5/150
1094/1094 [==============================] - 3s 3ms/step - loss: 0.0014 - val_loss: 5.8400e-04
Epoch 6/150
1094/1094 [==============================] - 3s 3ms/step - loss: 0.0015 - val_loss: 4.1466e-04
Epoch 7/150
1094/1094 [==============================] - 3s 3ms/step - loss: 0.0014 - val_loss: 4.6483e-04
Epoch 8/150
1094/1094 [==============================] - 3s 3ms/step - loss: 0.0017 - val_loss: 5.2113e-04
Epoch 9/150
1094/1094 [==============================] - 3s 3ms/step - loss: 0.0014 - val_loss: 4.2990e-04
Epoch 10/150
1094/1094 [=================

### Testing the autoencoder

In [ ]:
def predict(x, threshold=0.074, window_size=32):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (2150, 115)
Detected anomalies: 1.557338367154318%

1
Shape of data: (58669, 115)
Detected anomalies: 100.0%

2
Shape of data: (28305, 115)
Detected anomalies: 100.0%

3
Shape of data: (27698, 115)
Detected anomalies: 100.0%

4
Shape of data: (97783, 115)
Detected anomalies: 0.0%

5
Shape of data: (110617, 115)
Detected anomalies: 0.0%



## IOT Device 8: SimpleHome XCS7 1002 WHT Security Camera

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("8.benign.csv")
X_train = benign[:40000]
X_test0 = benign[40000:]
X_test1 = load_nbaiot("8.mirai.scan.csv")
X_test2 = load_nbaiot("8.mirai.ack.csv")
X_test3 = load_nbaiot("8.mirai.syn.csv")
X_test4 = load_nbaiot("8.mirai.udp.csv")
X_test5 = load_nbaiot("8.mirai.udpplain.csv")
X_test6 = load_nbaiot("8.gafgyt.combo.csv")
X_test7 = load_nbaiot("8.gafgyt.junk.csv")
X_test8 = load_nbaiot("8.gafgyt.scan.csv")
X_test9 = load_nbaiot("8.gafgyt.tcp.csv")
X_test10 = load_nbaiot("8.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(40000, 115) (6585, 115) (45930, 115) (111480, 115) (125715, 115) (151879, 115) (78244, 115) (54283, 115) (28579, 115) (27825, 115) (88816, 115) (103720, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.017), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=230,
    validation_split=0.3,
    shuffle=True,
    # callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/230
875/875 [==============================] - 3s 3ms/step - loss: 0.0055 - val_loss: 0.0020
Epoch 2/230
875/875 [==============================] - 3s 3ms/step - loss: 0.0028 - val_loss: 0.0023
Epoch 3/230
875/875 [==============================] - 2s 3ms/step - loss: 0.0155 - val_loss: 0.0618
Epoch 4/230
875/875 [==============================] - 3s 3ms/step - loss: 0.0163 - val_loss: 0.0052
Epoch 5/230
875/875 [==============================] - 3s 3ms/step - loss: 0.0064 - val_loss: 0.0035
Epoch 6/230
875/875 [==============================] - 2s 3ms/step - loss: 0.0059 - val_loss: 0.0033
Epoch 7/230
875/875 [==============================] - 2s 3ms/step - loss: 0.0098 - val_loss: 0.0163
Epoch 8/230
875/875 [==============================] - 3s 3ms/step - loss: 0.0197 - val_loss: 0.0213
Epoch 9/230
875/875 [==============================] - 3s 3ms/step - loss: 0.0205 - val_loss: 0.0213
Epoch 10/230
875/875 [==============================] - 2s 3ms/step - loss: 0.0205 - val_lo

### Testing the autoencoder

In [ ]:
def predict(x, threshold=0.056, window_size=23):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (6585, 115)
Detected anomalies: 13.941794910863933%

1
Shape of data: (45930, 115)
Detected anomalies: 100.0%

2
Shape of data: (111480, 115)
Detected anomalies: 100.0%

3
Shape of data: (125715, 115)
Detected anomalies: 100.0%

4
Shape of data: (151879, 115)
Detected anomalies: 100.0%

5
Shape of data: (78244, 115)
Detected anomalies: 100.0%

6
Shape of data: (54283, 115)
Detected anomalies: 100.0%

7
Shape of data: (28579, 115)
Detected anomalies: 100.0%

8
Shape of data: (27825, 115)
Detected anomalies: 100.0%

9
Shape of data: (88816, 115)
Detected anomalies: 100.0%

10
Shape of data: (103720, 115)
Detected anomalies: 100.0%



## IOT Device 9: SimpleHome XCS7 1003 WHT Security Camera

### Loading the dataset

In [ ]:
def load_nbaiot(filename):
    return np.genfromtxt(
        os.path.join("/drive/My Drive/Mini_Project_Dataset/N-Balot_Dataset", filename),
        delimiter=",",
        skip_header=1
    )

benign = load_nbaiot("9.benign.csv")
X_train = benign[:15000]
X_test0 = benign[15000:]
X_test1 = load_nbaiot("9.mirai.scan.csv")
X_test2 = load_nbaiot("9.mirai.ack.csv")
X_test3 = load_nbaiot("9.mirai.syn.csv")
X_test4 = load_nbaiot("9.mirai.udp.csv")
X_test5 = load_nbaiot("9.mirai.udpplain.csv")
X_test6 = load_nbaiot("9.gafgyt.combo.csv")
X_test7 = load_nbaiot("9.gafgyt.junk.csv")
X_test8 = load_nbaiot("9.gafgyt.scan.csv")
X_test9 = load_nbaiot("9.gafgyt.tcp.csv")
X_test10 = load_nbaiot("9.gafgyt.udp.csv")

In [ ]:
print(X_train.shape, X_test0.shape, X_test1.shape, X_test2.shape,
      X_test3.shape, X_test4.shape, X_test5.shape, X_test6.shape, 
      X_test7.shape, X_test8.shape, X_test9.shape, X_test10.shape)

(15000, 115) (4528, 115) (43674, 115) (107187, 115) (122479, 115) (157084, 115) (84436, 115) (59398, 115) (27413, 115) (28572, 115) (98075, 115) (102980, 115)


### Training the autoencoder

In [ ]:
scaler = MinMaxScaler()
x = scaler.fit_transform(X_train)

ae = Autoencoder()
ae.compile(optimizer=Adam(learning_rate=0.006), loss='mse')
monitor = EarlyStopping(
    monitor='val_loss',
    min_delta=1e-9,
    patience=5,
    verbose=1,
    mode='auto'
)
ae.fit(
    x=x,
    y=x,
    epochs=500,
    validation_split=0.3,
    shuffle=True,
    # callbacks=[monitor]
)

training_loss = losses.mse(x, ae(x))
threshold = np.mean(training_loss)+np.std(training_loss)

Epoch 1/500
329/329 [==============================] - 2s 3ms/step - loss: 0.0076 - val_loss: 0.0019
Epoch 2/500
329/329 [==============================] - 1s 3ms/step - loss: 0.0019 - val_loss: 0.0017
Epoch 3/500
329/329 [==============================] - 1s 3ms/step - loss: 0.0016 - val_loss: 0.0012
Epoch 4/500
329/329 [==============================] - 1s 3ms/step - loss: 0.0013 - val_loss: 0.0012
Epoch 5/500
329/329 [==============================] - 1s 3ms/step - loss: 0.0013 - val_loss: 0.0010
Epoch 6/500
329/329 [==============================] - 1s 3ms/step - loss: 0.0011 - val_loss: 0.0011
Epoch 7/500
329/329 [==============================] - 1s 3ms/step - loss: 0.0013 - val_loss: 9.8747e-04
Epoch 8/500
329/329 [==============================] - 1s 3ms/step - loss: 0.0011 - val_loss: 0.0010
Epoch 9/500
329/329 [==============================] - 1s 3ms/step - loss: 0.0011 - val_loss: 0.0014
Epoch 10/500
329/329 [==============================] - 1s 3ms/step - loss: 0.0016 - va

### Testing the autoencoder

In [ ]:
def predict(x, threshold=0.004, window_size=25):
    x = scaler.transform(x)
    predictions = losses.mse(x, ae(x)) > threshold
    # Majority voting over `window_size` predictions
    return np.array([np.mean(predictions[i-window_size:i]) > 0.5
                     for i in range(window_size, len(predictions)+1)])

test_data = [X_test0, X_test1, X_test2, X_test3, X_test4, X_test5, X_test6, X_test7, X_test8, X_test9, X_test10]

for i, x in enumerate(test_data):
    print(i)
    outcome = predict(x)
    print_stats(x, outcome)

0
Shape of data: (4528, 115)
Detected anomalies: 4.174067495559503%

1
Shape of data: (43674, 115)
Detected anomalies: 100.0%

2
Shape of data: (107187, 115)
Detected anomalies: 100.0%

3
Shape of data: (122479, 115)
Detected anomalies: 100.0%

4
Shape of data: (157084, 115)
Detected anomalies: 100.0%

5
Shape of data: (84436, 115)
Detected anomalies: 100.0%

6
Shape of data: (59398, 115)
Detected anomalies: 100.0%

7
Shape of data: (27413, 115)
Detected anomalies: 100.0%

8
Shape of data: (28572, 115)
Detected anomalies: 100.0%

9
Shape of data: (98075, 115)
Detected anomalies: 100.0%

10
Shape of data: (102980, 115)
Detected anomalies: 100.0%

